In [2]:
using Pkg
Pkg.activate(".")
using Flux
using SignalAnalysis
using WAV
using Plots
using MFCC
using DSP
using JLD2

DATA = "datasets/audioMNIST"
PREPROCESSED = "preprocessed/audioMNIST"
RESAMPLED = "$PREPROCESSED/resampled"
TRAINFILE = "$PREPROCESSED/train.jld2"
NFFT=512
NOVERLAP=128


  Activating project at `~/julia-ml`


128

In [2]:

spec = Spectrogram(nfft=NFFT, noverlap=NOVERLAP, window=hamming)

# spec = spectrogram(sample[:, 1], NFFT, NOVERLAP; fs, window=hanning)
# @info mfcc(sample, fs)

# sample[1:3:end, :]

responsetype = Lowpass(8000; fs=48000)
designmethod = Butterworth(4)
dofilt(x) = filt(digitalfilter(responsetype, designmethod), x)


for sname in Base.Filesystem.readdir(DATA; join=true)
    for fname in Base.Filesystem.readdir(sname)
        path = "$(sname)/$(fname)"
        sample, fs = wavread(path)
        resampled = dofilt(sample[1:3:end, 1])
        resampled = resampled ./ maximum(abs.(resampled))
        resampled = vcat(resampled, .001*randn(16000 - size(resampled, 1)))
        wavwrite(resampled, "$RESAMPLED/$fname"; Fs=fs/3)
    end
end


In [3]:

result = zeros(98, 13, 60, 50, 10) # TODO don't hardcode 50 samples per digit

count = 0
for (i, fname) in enumerate(Base.Filesystem.readdir(RESAMPLED))
    path = "$(RESAMPLED)/$(fname)"
    sample, fs = wavread(path)

    count += 1

    
    digit = parse(Int, fname[1])
    sampleidx = parse(Int, fname[6:(end-4)]) 
    speakeridx = parse(Int, fname[3:4])


    m = mfcc(sample)[1]
    # display(maximum(abs.(sample)))
    # display(mfcc(sample))

    result[:, :, speakeridx, sampleidx+1, digit+1] = m

    # result[:, sampleidx+1, digit+1] = vcat(sample, zeros(16000 - size(sample, 1)))
end


# result = result ./ maximum(result)

jldsave("$TRAINFILE"; samples=result)
count

# jldopen("$PREPROCESSED/samples.jld2")

# size(mfcc(result, 16000))


# size(b)

30000

In [4]:
v = jldopen(TRAINFILE, "r") do f
    f["samples"]
end


v[:, :, 1, 1, 1]





98×13 Matrix{Float64}:
 -1.31879    11.8812   -35.0504   …   -1.75098    6.15961     0.784658
 -0.104769   14.3677   -29.9443        8.31677    6.54818    -2.72692
 -0.789199   11.9132   -33.069         0.702421   3.75784     8.26592
  3.95704    10.4969   -32.6097        0.503208  -3.96923     1.67991
  8.4115      8.35469  -43.0553       -2.60654   -3.62558    -1.29019
 14.1012      4.07182  -38.4999   …   -9.44099    0.592751   -1.0469
 19.2323      1.8675   -36.8761       -8.40009    2.64764     1.96717
 20.6748     -5.07048  -37.4907       -1.71315    9.7601      0.874098
 21.2881     -5.77884  -37.5579       -9.17374   10.3758     -8.85956
 22.8429     -7.64979  -36.907       -14.9303    13.4636     -5.85144
  ⋮                               ⋱    ⋮                    
  5.29723   -40.2072    -7.11785       1.47991   -2.7468      2.45945
  5.64516   -40.7676    -8.46019  …    1.12471    1.16182     4.66058
  5.98654   -39.4755    -6.41862      -2.57309    3.1427      1.94474
  6.1